In [90]:
#import das bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sys
import analise_exploratoria as ae
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler


In [91]:
# Leitura dos dados
data = pd.read_csv('datasets/RHP_data.csv')

# Visualização dos dados
data.head()

,Id,Peso,Altura,IMC,Atendimento,DN,IDADE,Convenio,PULSOS,PA SISTOLICA,PA DIASTOLICA,PPA,B2,SOPRO,FC,HDA 1,HDA2,SEXO,MOTIVO1,MOTIVO2
0,1,5.0,51,19.0,11/05/06,30/03/06,0.12,GS,Normais,NaN,NaN,Não Calculado,Normal,Sistólico,112,Palpitacao,NaN,M,6 - Suspeita de cardiopatia,6 - Palpitação/taquicardia/arritmia
1,2,3.5,50,14.0,25/05/05,19/05/05,0.02,GS,Normais,NaN,NaN,Não Calculado,Normal,ausente,128,Dispneia,NaN,M,6 - Suspeita de cardiopatia,6 - Dispnéia
2,3,0.0,0,NaN,12/06/01,08/05/05,-4.05,SULA,Normais,NaN,NaN,Não Calculado,Normal,Sistólico,88,Assintomático,NaN,M,2 - Check-up,NaN
3,4,8.1,65,19.0,15/10/09,21/04/09,0.5,NaN,Normais,NaN,NaN,Não Calculado,Normal,ausente,92,Assintomático,NaN,M,5 - Parecer cardiológico,NaN
4,5,39.5,142,20.0,23/01/09,24/10/99,9.6,Sulam,Normais,85.0,50.0,Normal,Normal,ausente,90,NaN,NaN,M,5 - Parecer cardiológico,5 - Atividade física


In [92]:
# padronização dos nomes das colunas
ae.padronizar_colunas(data)

In [93]:
#dropar colunas que não serão utilizadas
data.drop(columns=['atendimento', 'dn','convenio','hda_2'], inplace=True)

In [94]:
#colunas numericas
cols_numericas = ['peso', 'altura','idade', 'imc', 'pa_sistolica', 'pa_diastolica'] #fc tratar depois

#ae.tratar_dados_nulos(data, cols_numericas) todo: implementar essa função

for col in cols_numericas:
    data[col] = data[col].apply(lambda x: 0 if x == '#VALUE!' else float(x))
    data[col] = data[col].apply(lambda x: 0 if x < 0 else x)
    data[col] = data[col].fillna(0)

    
data[cols_numericas].isnull().sum()

peso             0
altura           0
idade            0
imc              0
pa_sistolica     0
pa_diastolica    0
dtype: int64

In [95]:
## TODO: implementar essa função
#colunas categoricas
cols_categoricas = [ 'pulsos', 'ppa', 'b2', 'sexo', 'sopro', 'hda_1', 'motivo_1', 'motivo_2']

for col in cols_categoricas:
    data[col] = data[col].replace('#VALUE!', pd.NA)
    data[col] = data[col].fillna('Nao informado')
    data[col] = data[col].apply(lambda x: x.lower())

# em sexo, substituir 'masculino' por 'm' e 'feminino' por 'f'
data['sexo'] = data['sexo'].replace('masculino', 'm')
data['sexo'] = data['sexo'].replace('feminino', 'f')



data[cols_categoricas].isnull().sum()

pulsos      0
ppa         0
b2          0
sexo        0
sopro       0
hda_1       0
motivo_1    0
motivo_2    0
dtype: int64

In [96]:
# vendo os tipos das colunas
data.dtypes

id                 int64
peso             float64
altura           float64
imc              float64
idade            float64
pulsos            object
pa_sistolica     float64
pa_diastolica    float64
ppa               object
b2                object
sopro             object
fc                object
hda_1             object
sexo              object
motivo_1          object
motivo_2          object
dtype: object

In [97]:
# Transforming 'fc' column to float
# If 'fc' has a format like 'number a number', take the first number
data['fc'] = data['fc'].fillna(0)  # Replace NaN with 0
data['fc'] = data['fc'].astype(str).str.extract(r'(\d+)')[0]  # Extract the first number
data['fc'] = data['fc'].astype(float)  # Convert to float

### preenchendo as colunas 0 com a mediana

In [98]:
#preenchendo as colunas 0 com a mediana
for col in cols_numericas:
    data[col] = data[col].apply(lambda x: data[col].median() if x == 0 else x)

### preenchendo nao informado com a moda

In [99]:
#preenchendo nao informado com a moda
for col in cols_categoricas:
    data[col] = data[col].apply(lambda x: data[col].mode()[0] if x == 'nao informado' else x)

### separando os dados em treino e teste

In [100]:
# separando os dados em treino e teste
df_test = pd.read_csv('test.csv')
df_train = pd.read_csv('train.csv')

#trocando o nome da coluna Id para id em df_test e df_train
df_test.rename(columns={'Id':'id'}, inplace=True)
df_train.rename(columns={'Id':'id'}, inplace=True)

#dando merge de data com df_train usando o id, mas mantendo a ordem de ids de df_train
data_train = pd.merge(df_train, data, on='id', how='inner')

#dando merge de data com df_test usando o id, mas mantendo a ordem de ids de df_test
data_test = pd.merge(df_test, data, on='id', how='inner')

In [101]:
data_train.head()

,id,CLASSE,peso,altura,imc,idade,pulsos,pa_sistolica,pa_diastolica,ppa,b2,sopro,fc,hda_1,sexo,motivo_1,motivo_2
0,10524,Normal,28.0,123.0,19.0,5.79,normais,110.0,80.0,has-1 pad,normal,ausente,100.0,palpitacao,m,6 - suspeita de cardiopatia,6 - palpitação/taquicardia/arritmia
1,17249,Anormal,32.0,99.0,16.0,12.11,normais,100.0,60.0,não calculado,normal,sistólico,100.0,assintomático,m,5 - parecer cardiológico,nao informado
2,4483,Anormal,13.0,87.0,17.0,2.21,normais,90.0,50.0,não calculado,normal,sistólico,100.0,assintomático,m,6 - suspeita de cardiopatia,6 - sopro
3,6830,Normal,11.0,99.0,16.0,1.60,normais,90.0,50.0,não calculado,normal,ausente,100.0,assintomático,f,1 - cardiopatia já estabelecida,1 - cardiopatia congenica
4,11652,Normal,42.4,136.0,23.0,0.06,normais,100.0,60.0,não calculado,normal,ausente,78.0,assintomático,f,5 - parecer cardiológico,nao informado


In [102]:
#removendo outliers
def remove_outliers(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    df = df[(df[col] <= (Q3 + 1.5 * IQR)) & (df[col] >= (Q1 - 1.5 * IQR))]
    return df

In [103]:
for col in cols_numericas:
    data_train = remove_outliers(data_train, col)

In [104]:
data_train.shape

(11398, 17)

In [105]:
#transformando as colunas categoricas em dummies
data_train = pd.get_dummies(data_train, columns=cols_categoricas)

### testar modelos

In [106]:
#separando x_train e y_train
x_train = data_train.drop(columns=['id', 'CLASSE'])
y_train = data_train['CLASSE']

In [107]:
#vendo se tem algum valor nulo em x_train e y_train
x_train.isnull().sum()

peso                                            0
altura                                          0
imc                                             0
idade                                           0
pa_sistolica                                    0
                                               ..
motivo_2_6 - has/dislipidemia/obesidade         0
motivo_2_6 - palpitação/taquicardia/arritmia    0
motivo_2_6 - sopro                              0
motivo_2_nao informado                          0
motivo_2_outro                                  0
Length: 63, dtype: int64

In [111]:
#usando dummies para as colunas categoricas de y_train
y_train = pd.get_dummies(y_train)

y_train.value_counts()

Anormal  Normal
False    True      6302
True     False     4026
False    False     1070
Name: count, dtype: int64

In [113]:
#remover as linhas onde Normal e Anormal for false



ValueError: No group keys passed!